In [33]:
import numpy as np
#import scipy
import scipy.linalg as la
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns
#from mnist import MNIST
#import pickle
#import os
sns.set()
sns.set_style("ticks")
np.random.seed(0)
#np.warnings.filterwarnings('ignore')
print("Modules Loaded")


def load_data(n=30):
	np.random.seed(0)
	x = np.random.uniform(0, 1, n).reshape(n, 1)
	fx = 4* np.sin(np.pi * x) * np.cos(6 * np.pi * x**2)
	e = np.random.randn(n).reshape(n, 1)
	y = fx + e
	#print(x.shape, fx.shape, e.shape, y.shape)
	return(x, fx, y)


def train(K, y, L=10**-6):
	# alpah = (HH^T + lambda * I )^-1 y 
	# a x + b
	#a = np.inner(H, H.T) + L * np.identity(H.shape[0]) 
	a = K + L * np.identity(K.shape[0]) 
	alpha = la.solve( a, y )
	return(alpha)


def predict(K, alpha):
	f = K.dot(alpha)
	return(f)

def MSE(f, y):
	mse = ((f-y)**2).mean()
	return(mse)

def poly(x, z, d):
	k = (1 + x.T.dot(z) )**d
	return(k)

def rbf(x,z,gamma):
	norm = np.linalg.norm(x-z, ord=2)**2
	k = np.exp( -gamma * norm)
	return(k)

def makeKernel(X, kernel, X2=None, hyper=100):
	if(X2 is None):
		X2 = X	
	n = X.shape[0]
	m = X2.shape[0]
	K = np.zeros((m,n))
	for i in range(m):
		x = X2[i,:]
		for j in range(n):
			z = X[j,:]
			K[i, j] = kernel(x,z,hyper)
	return(K)


Modules Loaded


In [19]:
def kfold(K, y, k = 5):
	idx = np.random.permutation(K.shape[0])
	K_trains = []
	K_vals = []
	y_trains = []
	y_vals = []
	for i in range(k):
		start = int(i*X.shape[0]/k)
		end = int((i+1)*X.shape[0]/k)
		idx_val = idx[start:end]
		idx_train = np.concatenate( (idx[0:start], idx[end:]) )
		# select columns and rows not used by validation
		K_trains.append(  K[idx_train, :][:, idx_train]  ) 
		# select only rows used for validation and then reomove columns no longer used
		K_vals.append(K[idx_val, :][:, idx_train])

		y_trains.append(y[idx_train, :])
		y_vals.append(y[idx_val, :])

	return(K_trains, y_trains, K_vals, y_vals)
def DoCV(X, fx, y, kernel, k=10):
	n = X.shape[0]	
	
	if(kernel == rbf):
		hypers = np.float_power( 10, np.arange(-3, 4, .25) )
	elif(kernel == poly):
		hypers = np.arange(1, 100, 2)
	#print(hypers)
	Ls = np.float_power( 10, np.arange(-10, 5, .25) )
	
	results = []
	for hyper in hypers:	
		K = makeKernel(X, kernel, hyper = hyper)
		# leave one out cross validation
		K_trains, y_trains, K_vals, y_vals =  kfold(K, y, k = k)
		for L in Ls:
			mses = []
			for i in range(k):
				K_train = K_trains[i]; y_train = y_trains[i]; K_val = K_vals[i]; y_val = y_vals[i]
				#print(K_train.shape, y_train.shape, K_val.shape, y_val.shape)
				alpha = train(K_train, y_train, L=L)
				f = predict(K_val, alpha)
				mse = MSE(f, y_val)
				mses.append(mse)
			results.append( ( np.mean(mses), hyper, L ) )

	best = np.inf; bestidx = 0
	for idx, line in enumerate(results):
		if(line[0] < best):
			best = line[0]
			bestidx = idx
	mse, hyper, L = results[bestidx]
	print(mse, hyper, L)
	return(hyper, L,mse)



In [20]:
X,fx,y = load_data(40)
#K = makeKernel(x, rbf, X2=None, hyper=100)
#K_trains, y_trains, K_vals, y_vals = kfold(K, y, k = 5)
hyper, L, mse = DoCV(X, fx, y, poly, k=10)

/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=7.86463e-17): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=7.82785e-17): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=6.66252e-17): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=6.91753e-17): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=9.24256e-17): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=8.33828e-17): result may not be accurate

/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=2.39174e-19): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=1.35972e-19): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=6.80915e-19): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.58531e-19): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=6.09759e-20): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.74122e-19): result may not be accurate

/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=1.71107e-22): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.296e-21): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.04283e-21): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=7.3344e-21): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.53599e-21): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=6.99312e-21): result may not be accurate.
/

/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.96394e-18): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=6.21451e-18): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=6.61622e-18): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=7.22128e-18): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=8.73923e-18): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=9.75038e-18): result may not be accurate

/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=3.5624e-24): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=7.56774e-24): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=1.01167e-23): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=3.10058e-24): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=5.00268e-24): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.83498e-24): result may not be accurate.

...

/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=9.83432e-34): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.30522e-33): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=2.11901e-33): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=1.12555e-34): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=1.61476e-33): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=3.22248e-33): result may not be accurate

/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=2.20284e-27): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=2.67012e-27): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=3.56659e-27): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=2.1004e-27): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.45103e-27): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=1.14946e-27): result may not be accurate.

/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.52521e-37): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.79113e-37): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=4.87308e-37): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=7.73361e-38): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=7.15045e-38): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=1.3355e-36): result may not be accurate.

1.4366985134161308 59 31.622776601683793


/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=2.35301e-34): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=6.38518e-34): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=8.78044e-35): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=8.05654e-35): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=6.64978e-36): result may not be accurate.
/Users/yyan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: LinAlgWarning: Ill-conditioned matrix (rcond=3.20786e-34): result may not be accurate

In [21]:
mse

1.4366985134161308

In [15]:
def plot( x,f,y,kernel,best_param,best_L):
    plt.clf()
    K_poly = form_kernel(x,kernel,best_param,x2=None)
    a = train(K_poly,y,best_L)
    x_plot = np.arange(0,1,0.001)
    x_plot =  x_plot.reshape(x_plot.shape[0],1)
    K_plot = form_kernel(x_plot,kernel,best_param,x2=x)
    y_pred = pred(K_plot,a)
    f_plot = 4*np.sin(np.pi*x_plot)*np.cos(6*np.pi*x_plot**2)
    x, p5, p95 = bootstrap(x, y, kernel,best_param, best_L, B=300)
    sns.set()
    plt.plot(x,y,'.',label ='y')
    plt.plot(x_plot,f_plot,'-',label = 'f')
    plt.plot(x_plot,y_pred,label = 'f_{}'.format(kernel))
    #plt.plot(x_plot,y_pred_rbf_best,label = 'frbf')
    plt.plot(x_plot,p5,label = 'p5')
    plt.plot(x_plot,p95,label = 'p95')
    #plt.fill_between(x_plot[:,0], p5, p95, color='grey', alpha=0.5)
    plt.title("kernel plot")
    plt.legend()
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.ylim(-5, 5)


In [ ]:
x,f,y = generate_dataset(30)
param_list= np.arange(0, 20,2)
L_list = np.float_power( 10,np.arange(-10, 5))
#loo(x,y,[2,3],[3,4],poly)
best_param_poly,best_L_poly, min_mse_poly  =cv(x,y,param_list,L_list,poly,10)
#best_param_rbf,best_L_rbf, min_mse_rbf  = cv(x,y,param_list,L_list,rbf,10)
print('best_L_poly:{}, best_param_poly:{}, min_mse_poly:{}'
      .format(best_L_poly, best_param_poly,min_mse_poly))
#print('best_L_rbf:{}, best_param_rbf:{}, min_mse_rbf:{}'
      #.format(best_L_rbf, best_param_rbf,min_mse_rbf))
makeKernel(X, kernel, X2=None, hyper=100):

In [34]:
x,f,y = load_data(30)
x2,f2,y2 = load_data(10)
K = makeKernel(x,poly,X2=None,hyper = 3)
a = train(K, y, L=0.1)
kx = makeKernel(x,poly,X2= x2,hyper = 3)
f = predict(kx, a)

In [39]:
f

array([[0.19647727],
       [0.23403454],
       [0.21484323],
       [0.19500216],
       [0.14701626],
       [0.22596563],
       [0.15236871],
       [0.16922658],
       [0.09381106],
       [0.13266536]])

In [ ]:
def kfold(K, y, k = 5):
	idx = np.random.RandomState(seed=42).permutation(K.shape[0])
	K_trains = []
	K_vals = []
	y_trains = []
	y_vals = []
	for i in range(k):
		start = int(i*X.shape[0]/k)
		end = int((i+1)*X.shape[0]/k)
		idx_val = idx[start:end]
		idx_train = np.concatenate( (idx[0:start], idx[end:]) )
		# select columns and rows not used by validation
		K_trains.append(  K[idx_train, :][:, idx_train]  ) 
		# select only rows used for validation and then reomove columns no longer used
		K_vals.append(K[idx_val, :][:, idx_train])

		y_trains.append(y[idx_train, :])
		y_vals.append(y[idx_val, :])

	return(K_trains, y_trains, K_vals, y_vals)


def plots(X, fx, y, kernel, hyper, L, x, p5, p95):
	K = makeKernel(X, kernel, hyper = hyper)
	alpha = train(K, y, L=L)
	f = predict(K, alpha)
	mse = MSE(f, y)
	n = X.shape[0]

	name = "{}_{}.pdf".format(n, kernel.__name__)

	# redefine fx to use more points 
	fx = 4* np.sin(np.pi * x) * np.cos(6 * np.pi * x**2)

	sns.scatterplot(X[:,0], y[:,0], color="green", label="y_i")
	sns.lineplot(x[:,0], fx[:,0], label="f(x)")
	sns.lineplot(X[:,0], f[:,0], label="f_hat")
	plt.title(kernel.__name__)
	plt.legend()	
	plt.xlabel("x")
	plt.ylabel("f(x)")
	plt.ylim(-4.5, 6.5)	
	# add boostrap 		
	#sns.lineplot(x[:,0], p5, label="5 conf")
	#sns.lineplot(x[:,0], p95, label="95 conf")
	plt.fill_between(x[:,0], p5, p95, color='grey', alpha=0.5)
	plt.savefig(name)
	
	plt.clf()
	print(name, mse)

def DoCV(X, fx, y, kernel, k=10):
	n = X.shape[0]	
	
	if(kernel == rbf):
		hypers = np.float_power( 10, np.arange(-3, 4, .25) )
	elif(kernel == poly):
		hypers = np.arange(1, 100, 2)
	#print(hypers)
	Ls = np.float_power( 10, np.arange(-10, 5, .25) )
	
	results = []
	for hyper in hypers:	
		K = makeKernel(X, kernel, hyper = hyper)
		# leave one out cross validation
		print("K.shape{}")
		K_trains, y_trains, K_vals, y_vals =  kfold(K, y, k = k)
		for L in Ls:
			mses = []
			for i in range(k):
				K_train = K_trains[i]; y_train = y_trains[i]; K_val = K_vals[i]; y_val = y_vals[i]
				print(K_train.shape, y_train.shape, K_val.shape, y_val.shape)
				alpha = train(K_train, y_train, L=L)
				f = predict(K_val, alpha)
				mse = MSE(f, y_val)
				mses.append(mse)
			results.append( ( np.mean(mses), hyper, L ) )

	best = np.inf; bestidx = 0
	for idx, line in enumerate(results):
		if(line[0] < best):
			best = line[0]
			bestidx = idx
	mse, hyper, L = results[bestidx]
	print(mse, hyper, L)
	return(hyper, L)

def bootstrap(X, y, kernel, hyper, L, B=300):
	n = X.shape[0]
	# make all xs to predict on
	step = .01
	x = np.arange(0, 1 + step, step)
	x = x.reshape(x.shape[0], 1)
	
	testn = 40
	
	fs = np.zeros((B, x.shape[0]))
	for i in range(B):
		if(i % 100 == 0 ):
			print("bootstrap", i)
		idxs = np.random.choice(n, n)
		Xb = X[idxs,:]
		yb = y[idxs,:]
		K = makeKernel(Xb, kernel, hyper = hyper)
		alpha = train(K, yb, L=L)
		
		kx = makeKernel(Xb, kernel, X2=x, hyper = hyper)
		f = predict(kx, alpha)
		#print(f[testn,0])
		#print(kx.shape, alpha.shape, f.shape)
		fs[i, :] = f[:,0]
	#print(fs.shape)
	
	p5 = np.percentile(fs, 5, axis=0)
	p95 = np.percentile(fs, 95, axis=0)

	#print(fs.shape)
	#print(np.sort(fs[:,testn]))
	#print(p5[testn], p95[testn], x[testn])
	return(x, p5, p95)



# find best hyper parameters 

hypers = [177.82794100389228, 47, 5.6234, 41]
Ls = [0.1, 0.316277, 1.778*(10**-12), 0.017782]
kernels = [rbf, poly, rbf, poly]
#hypers = None

i=0
for n in [30, 300]:
	X, fx, y = load_data(n = n)
	k = X.shape[0]
	if(k > 30):
		k = 10
	for kernel in [rbf, poly]:
		if(hypers is None):
			hyper, L = DoCV(X, fx, y, kernel, k=k)
		else:
			hyper = hypers[i]; L = Ls[i]
		print(hyper, L)
		# run bootstrap and plot 
		x, p5, p95 = bootstrap(X,y, kernel, hyper, L)
		plots(X, fx, y, kernel, hyper, L, x, p5, p95)
		i += 1





In [11]:
def DoCV(X, fx, y, kernel, k=10):
	n = X.shape[0]	
	if(kernel == rbf):
		#hypers = np.float_power( 10, np.arange(-3, 4, .25) )
		hypers = np.float_power( 10, np.arange(-3, 4, .25) )
	elif(kernel == poly):
		#hypers = np.arange(1, 100, 2)
		hypers = np.arange(1, 10)
	#print(hypers)
	Ls = np.float_power( 10, np.arange(-5, 2) )
	results = []
	for hyper in hypers:	
		K = makeKernel(X, kernel, hyper = hyper)
		# leave one out cross validation
		#print("K.shape{}")
		K_trains, y_trains, K_vals, y_vals =  kfold(K, y, k = k)
		for L in Ls:
			mses = []
			for i in range(k):
				K_train = K_trains[i]; y_train = y_trains[i]; K_val = K_vals[i]; y_val = y_vals[i]
				#print(K_train.shape, y_train.shape, K_val.shape, y_val.shape)
				alpha = train(K_train, y_train, L=L)
				f = predict(K_val, alpha)
				mse = MSE(f, y_val)
				mses.append(mse)
			results.append( ( np.mean(mses), hyper, L ) )

	best = np.inf; bestidx = 0
	for idx, line in enumerate(results):
		if(line[0] < best):
			best = line[0]
			bestidx = idx
	mse, hyper, L = results[bestidx]
	#print(mse, hyper, L)
	return(mse,hyper, L)

In [15]:
X, fx, y = load_data(n=30)
DoCV(X, fx, y, poly, k=30)

(3.087598555126692, 7, 0.0001)

In [14]:
def kfold(K, y, k = 5):
	idx = np.random.permutation(K.shape[0])
	K_trains = []
	K_vals = []
	y_trains = []
	y_vals = []
	for i in range(k):
		start = int(i*X.shape[0]/k)
		#print(start)
		end = int((i+1)*X.shape[0]/k)
		idx_val = idx[start:end]
		#print(idx_val.shape)
		idx_train = np.concatenate( (idx[0:start], idx[end:]) )
		#print(idx_train)
		# select columns and rows not used by validation
		K_trains.append(  K[idx_train, :][:, idx_train]  ) 
		# select only rows used for validation and then reomove columns no longer used
		K_vals.append(K[idx_val, :][:, idx_train])

		y_trains.append(y[idx_train, :])
		y_vals.append(y[idx_val, :])

	return(K_trains, y_trains, K_vals, y_vals)

In [ ]:
X, fx, y = load_data(n=30)
K = makeKernel(X, rbf, hyper = 0.1)
K_trains, y_trains, K_vals, y_vals =kfold(K, y, k = 5)

In [ ]:
K.shape

In [ ]:
K_vals[1].shape

In [ ]:
type(K)

In [ ]:
c =K[1,:][:,9]
c.shape

In [ ]:
K.shape

In [16]:
x

array([[0.78383235],
       [0.63483371],
       [0.24904309],
       [0.75807586],
       [0.31307694],
       [0.93723736],
       [0.04286545],
       [0.4408672 ],
       [0.91272229],
       [0.45500392],
       [0.50793434],
       [0.08490795],
       [0.42634979],
       [0.74571696],
       [0.8670814 ],
       [0.32366583],
       [0.1040744 ],
       [0.80210805],
       [0.39483195],
       [0.62776809],
       [0.03598854],
       [0.29990785],
       [0.04765322],
       [0.37221108],
       [0.2624152 ],
       [0.99052621],
       [0.39514535],
       [0.30753926],
       [0.22110114],
       [0.49402502],
       [0.05787519],
       [0.28970004],
       [0.23495039],
       [0.96343041],
       [0.61567387],
       [0.86258397],
       [0.09008526],
       [0.1489889 ],
       [0.67257496],
       [0.97899192]])